# MATH 565 — SciPy Sampling Demos

This notebook shows how to generate IID samples from specific distributions and visualize results.

## 0) `init.py`
This cell **writes** an `init.py` into the working directory, mirroring our class setup. If you already have an official `init.py` in your repo, feel free to delete this cell and use that file instead.

In [ ]:

# %%writefile init.py
from __future__ import annotations
import numpy as np
import scipy as sp
from scipy import stats, linalg
import matplotlib.pyplot as plt
# Statsmodels (for empirical CDF)
import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF

# Reproducibility helper
def seed_all(seed: int = 12345):
    np.random.seed(seed)

# Plot defaults
plt.rcParams.update({
    "figure.figsize": (6, 4),
    "axes.grid": True,
    "grid.alpha": 0.25,
})

# Simple shared color dictionary
colors = {
    "blue": "#1f77b4",
    "lightblue": "#9ecae1",
    "orange": "#ff7f0e",
    "green": "#2ca02c",
    "red": "#d62728",
    "purple": "#9467bd",
    "brown": "#8c564b",
    "pink": "#e377c2",
    "gray": "#7f7f7f",
}

# Helper: step-plot ECDF
def plot_ecdf(data, ax=None, label=None):
    ecdf = ECDF(data)
    if ax is None:
        import matplotlib.pyplot as plt
        ax = plt.gca()
    ax.step(ecdf.x, ecdf.y, where="post", label=label)
    ax.set_xlabel("x")
    ax.set_ylabel("F_n(x)")
    return ax


### Import the packages we used before (and our `init.py`)

In [ ]:

import sys, platform
from init import *  # imports numpy as np, scipy as sp, stats, plt, sm, ECDF, helpers

print(f"Python: {sys.version.split()[0]}")
print(f"Platform: {platform.platform()}")
print(f"NumPy: {np.__version__}")
print(f"SciPy: {sp.__version__}")
import matplotlib
print(f"Matplotlib: {matplotlib.__version__}")
try:
    import qmcpy as qp
    print(f"qmcpy: {qp.__version__}")
except Exception:
    print("qmcpy not found (that's OK here).")

seed_all(12345)


## 1) SciPy: IID Binomial samples (n = 3, p = 0.6)
- Generate samples with `scipy.stats.binom.rvs`
- Show **side-by-side** histograms for N = 10 and N = 1000
- Overlay the true PMF for comparison

In [ ]:

n, p = 3, 0.6
rv = stats.binom(n=n, p=p)
k = np.arange(n+1)
pmf = rv.pmf(k)

# Two sample sizes
N_small, N_large = 10, 1000
x_small = rv.rvs(size=N_small, random_state=np.random.default_rng(123))
x_large = rv.rvs(size=N_large, random_state=np.random.default_rng(456))

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
# Left: N=10
axes[0].hist(x_small, bins=np.arange(-0.5, n+1.5, 1), density=True, edgecolor="k", alpha=0.6)
axes[0].vlines(k, 0, pmf, colors="C1", lw=3, label="True PMF")
axes[0].set_title("Histogram (N = 10)")
axes[0].set_xlabel("k")
axes[0].set_ylabel("Density")
axes[0].legend()

# Right: N=1000
axes[1].hist(x_large, bins=np.arange(-0.5, n+1.5, 1), density=True, edgecolor="k", alpha=0.6)
axes[1].vlines(k, 0, pmf, colors="C1", lw=3, label="True PMF")
axes[1].set_title("Histogram (N = 1000)")
axes[1].set_xlabel("k")
axes[1].legend()

fig.suptitle(f"Binomial(n={n}, p={p}) — Histograms vs True PMF", y=1.02)
plt.tight_layout()
plt.show()


## 2) Zero-Inflated Exponential (ZIE)
- Probability of zero wait: $p_0 = 0.2$
- Positive waits are Exponential with **mean** 10 minutes (scale = 10)
- Generate IID samples; show **empirical CDF** using `statsmodels` for N = 10 and N = 1000

In [ ]:

p0 = 0.2                 # P(X = 0)
mean_pos = 10.0          # minutes
scale = mean_pos         # Exponential scale parameter
rng = np.random.default_rng(789)

def rvs_zero_inflated_expon(size, p_zero=0.2, scale=10.0, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    zeros = rng.random(size) < p_zero
    # Exponential samples for the positive part
    pos = rng.exponential(scale=scale, size=size)
    x = np.where(zeros, 0.0, pos)
    return x

N_small, N_large = 10, 1000
x_small = rvs_zero_inflated_expon(N_small, p_zero=p0, scale=scale, rng=rng)
x_large = rvs_zero_inflated_expon(N_large, p_zero=p0, scale=scale, rng=rng)

# Plot ECDFs side-by-side
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

plot_ecdf(x_small, ax=axes[0], label=f"N={N_small}")
axes[0].set_title("Zero-Inflated Exponential — ECDF (N=10)")
axes[0].legend()

plot_ecdf(x_large, ax=axes[1], label=f"N={N_large}")
axes[1].set_title("Zero-Inflated Exponential — ECDF (N=1000)")
axes[1].legend()

plt.tight_layout()
plt.show()


---
**Notes**
- Binomial samples used `scipy.stats.binom.rvs`.
- ZIE sampling generated zeros via a Bernoulli draw and positive times via `np.random.exponential(scale=10)`.
- ECDFs used `statsmodels.distributions.empirical_distribution.ECDF`.

_Last updated:_ 2025-08-24 21:34:57